Cargo los datos sobre temperaturas durante el año 2021 de Barcelona desde la estación de Drassanes que tiene un criterio de temperatura a nivel ciudad.

In [1]:
import pandas as pd

# Cargo los archivos CSV
df1 = pd.read_csv('temp_012021_062021.csv')
df2 = pd.read_csv('temp_072021_122021.csv')

# Concateno los dataFrames por filas
df_temp_2021 = pd.concat([df1, df2], axis=0, ignore_index=True)

# Muestro el dataFrame combinado
df_temp_2021.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2021-01-01,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"8,1","0,0","6,3",23:59,"9,9",00:50,8,"1,9","11,1",10:00,75,84,07:00,66,21:40
1,2021-01-02,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"7,0","0,0","4,0",07:00,"9,9",13:30,12,"2,2","8,3",22:00,64,77,23:00,55,Varias
2,2021-01-03,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"6,8","0,0","4,0",23:59,"9,7",13:20,21,"2,8","11,7",15:20,49,75,00:00,41,12:40
3,2021-01-04,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"6,3","0,0","2,5",02:40,"10,1",14:40,30,"2,8","10,0",13:50,62,78,05:50,46,Varias
4,2021-01-05,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"6,4","0,0","2,8",06:40,"10,0",13:00,33,"2,2","7,8",03:10,73,81,Varias,58,13:10


El dataframe está compuesto por 365 filas y 20 columnas

In [2]:
df_temp_2021.shape 

(365, 20)

Hay columnas que no contemplo utilizar por lo que decido eliminarlas para aligerar el dataframe.

In [3]:
df_temp_2021= df_temp_2021.drop(columns=['indicativo', 'nombre','hrMedia', 'hrMax', 'horaHrMax', 'hrMin', 'horaHrMin', 'altitud', 'dir', 'velmedia', 'racha', 'horaracha', 'horatmin', 'horatmax'])

In [4]:
df_temp_2021.head()

,fecha,provincia,tmed,prec,tmin,tmax
0,2021-01-01,BARCELONA,"8,1","0,0","6,3","9,9"
1,2021-01-02,BARCELONA,"7,0","0,0","4,0","9,9"
2,2021-01-03,BARCELONA,"6,8","0,0","4,0","9,7"
3,2021-01-04,BARCELONA,"6,3","0,0","2,5","10,1"
4,2021-01-05,BARCELONA,"6,4","0,0","2,8","10,0"


Reviso las características de cada columna, observo varios detalles sobre ellas que considero tratar:

* `fecha` es del tipo `object` y el formato debería ser `datetime`
* Hay varias columas que aparecen como tipo `object` utilizando una coma como decimal,  las transformaré como float con punto.


In [5]:
df_temp_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fecha      365 non-null    object
 1   provincia  365 non-null    object
 2   tmed       364 non-null    object
 3   prec       364 non-null    object
 4   tmin       364 non-null    object
 5   tmax       364 non-null    object
dtypes: object(6)
memory usage: 17.2+ KB


Realizo la transformación de la columna `fecha` al tipo `datetime`

In [6]:
df_temp_2021['fecha']= pd.to_datetime(df_temp_2021['fecha'], format='%Y-%m-%d')
print(df_temp_2021.dtypes)

fecha        datetime64[ns]
provincia            object
tmed                 object
prec                 object
tmin                 object
tmax                 object
dtype: object


Transformo las columnas 'tmed', 'prec', 'tmin', 'tmax' al tipo float, modificando el símbolo de coma por punto.

In [7]:
columnas_a_float = ['tmed', 'prec', 'tmin', 'tmax']

# Reemplazo las comas por puntos para que pueda realizarse el cambio al tipo float
for col in columnas_a_float:
    df_temp_2021[col] = df_temp_2021[col].str.replace(',', '.')  

df_temp_2021[columnas_a_float] = df_temp_2021[columnas_a_float].astype(float)

print(df_temp_2021.dtypes)

fecha        datetime64[ns]
provincia            object
tmed                float64
prec                float64
tmin                float64
tmax                float64
dtype: object


In [8]:
df_temp_2021.isnull().sum()

fecha        0
provincia    0
tmed         1
prec         1
tmin         1
tmax         1
dtype: int64

Analizo los valores nulos de la variable `tmed`; observo que aparecen varias columnas en ese registros con valores faltantes.

In [9]:
nulos_tmed = df_temp_2021['tmed'].isnull()
df_temp_2021[nulos_tmed]

,fecha,provincia,tmed,prec,tmin,tmax
174,2021-06-24,BARCELONA,NaN,0.0,NaN,NaN


La idea de la imputación en este caso es mantener la continuidad de la serie. Imputo los valores faltantes utilizando una función, que en caso de que encuentre un valor nulo en una columna, buscará el valor correspondiente en la fila anterior (día anterior). Si el valor anterior existe, se utiliza para reemplazar el valor nulo. 

In [10]:

# Ordeno el DataFrame por fecha para asegurar la correcta imputación
df_temp_2021 = df_temp_2021.sort_values('fecha')

# Selecciono las columnas a imputar 
columnas_a_imputar = ['tmed', 'tmin', 'tmax', 'prec']

# Se itera sobre las filas y las columnas a imputar
for index, row in df_temp_2021.iterrows():
    for columna in columnas_a_imputar:
        if pd.isna(row[columna]):
            # Buscar el valor del día anterior en la misma columna
            valor_anterior = df_temp_2021.loc[index - 1, columna]
            # Imputar si el valor anterior existe
            if not pd.isna(valor_anterior):
                df_temp_2021.loc[index, columna] = valor_anterior

Compruebo que los valores ha sido imputados totalmente.

In [11]:
df_temp_2021.isnull().sum()

fecha        0
provincia    0
tmed         0
prec         0
tmin         0
tmax         0
dtype: int64

In [12]:
df_temp_2021.head()

,fecha,provincia,tmed,prec,tmin,tmax
0,2021-01-01,BARCELONA,8.1,0.0,6.3,9.9
1,2021-01-02,BARCELONA,7.0,0.0,4.0,9.9
2,2021-01-03,BARCELONA,6.8,0.0,4.0,9.7
3,2021-01-04,BARCELONA,6.3,0.0,2.5,10.1
4,2021-01-05,BARCELONA,6.4,0.0,2.8,10.0


- Renombro la columna `provincia` por `ciudad`.
- Reordeno las columnas

In [13]:
df_temp_2021= df_temp_2021.rename(columns={'provincia': 'ciudad'})
df_temp_2021= df_temp_2021[['fecha', 'ciudad', 'tmin', 'tmax', 'tmed', 'prec']]
df_temp_2021.head()


,fecha,ciudad,tmin,tmax,tmed,prec
0,2021-01-01,BARCELONA,6.3,9.9,8.1,0.0
1,2021-01-02,BARCELONA,4.0,9.9,7.0,0.0
2,2021-01-03,BARCELONA,4.0,9.7,6.8,0.0
3,2021-01-04,BARCELONA,2.5,10.1,6.3,0.0
4,2021-01-05,BARCELONA,2.8,10.0,6.4,0.0


Exporto el dataframe a un archivo en formato `csv`

In [14]:
df_temp_2021.to_csv('temp_bcn_2021.csv', index= False)

Resumen del dataframe creado:

- Se eliminaron las columnas 'indicativo', 'nombre','hrMedia', 'hrMax', 'horaHrMax', 'hrMin', 'horaHrMin', 'altitud', 'dir', 'velmedia', 'racha', 'horaracha', 'horatmin', 'horatmax'.
- Se realizaron transformaciones de tipo en las columnas 'tmed', 'prec', 'tmin', 'tmax'.
- Se transformó la columna 'fecha' a tipo datetime
- Se trataron los valores nulos mediante la imputación del registro del día anterior al faltante, buscando mantener la continuidad de la serie. No hay valores faltantes.
- Se renombró la columna 'provincia' por 'ciudad'.
- Se exporto el dataframe a un archivo en formato csv.